# Dynamic circuits

In [ ]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.result import marginal_counts
from qiskit.visualization import *
from qiskit_aer import AerSimulator, Aer
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer.primitives import Sampler as AerSampler
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler
import numpy as np
from IPython.display import Image
# Ignore future warnings

with open('../..api_key.txt', 'r') as file:
    token = file.read()

import warnings
warnings.simplefilter(action='ignore')

**Dynamic circuits** are quantum circuits that contain mid-circuit measurements where the results of those measurements are used to condition quantum gates later in the circuit. The ability to condition future quantum operations on the classical measurement results is known as classical feedforward.

Dynamic circuits are quantum circuits that include control flow such as if statements and while loops

In [ ]:
qr = QuantumRegister(2)
cr = ClassicalRegister(2)
qc = QuantumCircuit(qr, cr)

q0, q1 = qr
b0, b1 = cr

qc.h(q0)

# Create a dynamic circuit that:
# 1) applies an x gate to q1 if the bit 0 is 0
# 1) applies an h gate to q1 if the bit 0 is 1

# Your code goes here:

qc.measure(q0, b0)




**Question:**

Considering the circuit above:

What are (roughly) the expected counts for the $|00\rangle$, $|01\rangle$, $|10\rangle$, $|11\rangle$ states?

In [ ]:
aer_backend = AerSimulator()

In [ ]:
# Execute the dynamic circuit (qc) on a local backend. Extract the counts.
service = QiskitRuntimeService(channel='ibm_quantum',token=token)
real_backend = service.backend('ibm_brisbane')
fake_backend = AerSimulator.from_backend(real_backend)
fake_backend

Transpile to ISA

In [ ]:
pm = generate_preset_pass_manager(backend=fake_backend, optimization_level=3)
isa_qc = pm.run(qc)
isa_qc.draw(idle_wires=False)

In [ ]:
with Session(service=service,backend=fake_backend) as session:
    sampler = Sampler(session=session)
    result = sampler.run([isa_qc]).result()

In [ ]:
counts = result.quasi_dists
counts

In [ ]:
plot_histogram(counts)

Note that the bar lables correspond to the binary representation of the states 
* 1 -> |01>
* 2 -> |10>
* 3 -> |11>

## Quantum Teleportation


Alice possesses a qubit in an unknown state $\lvert \psi \rangle$ and she wishes to transfer this quantum state to Bob. She can not simply clone or copy the state, but she can transfer the her qubit state to Bob.

By sending two bits of classical information Bob will at the end possess $\lvert \psi \rangle$, and Alice will no longer have it. This is known as teleportation.

The protocol requires three qubits:

1. The qubit to be teleported (Alice's qubit)
2. One half of an entangled pair of qubits (Alice's second qubit)
3. The other half of the entangled pair (Bob's qubit)

The protocol can be summarized in the following steps:

1. Create an entangled pair of qubits (Bell pair) shared between Alice and Bob.
2. Alice performs a Bell basis measurement on her two qubits.
3. Alice sends the classical results of her measurement to Bob.
4. Bob applies appropriate quantum gates based on Alice's measurement results to obtain the teleported state.

In [ ]:
Image(filename='images/quantum_teleportation.jpeg') 

In [ ]:
qr = QuantumRegister(3, name="q")
cr = ClassicalRegister(3, name="c")
s, a, b = qr
c0, c1, c2 = cr

In [ ]:
def create_bell_pair(qr: QuantumRegister, cr: ClassicalRegister) -> QuantumCircuit:
    """Creates a bell pair between qubits a and b."""
    qc = QuantumCircuit(qr, cr)
    # the first qubit is s but we won't be using it in this exercise
    s, a, b = qr
    # Create a bell pair between alice and bob.
    
    # Your code goes here:
    
    return qc

In [ ]:
def alice_gates(qr: QuantumRegister, cr: ClassicalRegister) -> QuantumCircuit:
    """Creates Alices's gates"""
    qc = QuantumCircuit(qr, cr)
    s, a, b = qr
    # Perform a bell basis measurement on alices qubits (s,a)
    qc.cx(s,a)
    qc.h(s)
    return qc

In [ ]:
def measure_and_send(qr: QuantumRegister, cr: ClassicalRegister):
    """Measures qubits a & b and 'sends' the results to Bob"""
    qc = QuantumCircuit(qr, cr)
    s, a, b = qr
    c0, c1, c2 = cr
    qc.measure([a,s],[c0,c1])
    return qc

In [ ]:
def bob_gates(qr: QuantumRegister, cr: ClassicalRegister):
    """Uses qc.if_test to control which gates are dynamically added"""
    qc = QuantumCircuit(qr, cr)
    s, a, b = qr
    c0, c1, c2 = cr
    # If the bits are `00`, no action is required.
    # If they are `01`, an 𝑋 gate (also known as a Pauli-X or a bit-flip gate) should be applied.
    # For bits `10`, a 𝑍 gate (also known as a Pauli-Z or a phase-flip gate) should be applied. 
    # Lastly, if the classical bits are `11`, a combined 𝑍𝑋 sequence should be applied.

    # Your code goes here: 
    
    return qc

In [ ]:
# Compose a circuit (inplace) with the name 'teleport' that incorporates the following steps:
# 1) create a bell pair
# 2) apply alices gates
# 3) measure and send
# 4) apply bobs gates
# 5) measure

# Your code goes here:



In [ ]:
# define source qubit
source = QuantumCircuit(qr,cr)
source.ry(np.pi/4,0)
source.draw()

In [ ]:
teleport_source = source.compose(teleport)
teleport_source.draw()

In [ ]:
# run job source 
source.measure_all()
counts_source = aer_backend.run(source, shots=4000).result().get_counts()

In [ ]:
plot_histogram(counts_source)

In [ ]:
pm = generate_preset_pass_manager(backend=real_backend,optimization_level=3)
isa_teleport_source = pm.run(teleport_source)

In [ ]:
# run teleport source on ibmq backend
job_teleport_source = real_backend.run(isa_teleport_source, dynamic=True)
job_teleport_source

In [ ]:
# Retrieve your results as soon as the job is finished

# counts_teleport_source = service.job("csy1a48yn5c0008bx5p0").result().get_counts()
# counts_teleport_source

In [ ]:
plot_histogram(counts_teleport_source)

In [ ]:
bobs_counts = marginal_counts(counts_teleport_source, [qr.index(b)])
plot_histogram(bobs_counts)